In [231]:
import pandas as pd
import numpy as np
import time
from timeit import timeit

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 1000)

In [232]:
LAST_CHECKED_SCORE = 18

In [233]:
%load_ext snakeviz  

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


# Steps
1. empty hand with 1.0 chance
+ or other hands: determine whih step to take next (e.g. plyer finished, split hand etc.)
2. deal random cards with chances
+ combine same hands (5,8 and 8,5)
+ finish blackjacks
3. get all possible moves for player, keep the drawn cards saved
+ vectorized adding (with diag)
+ [FALSE] aggregate bust hands (e.g. a 18 only needs 11,2,3 and 4+ can be immediately seen as a bust hand)
4. get all possible moves for dealer
+ aggregate deck and dealer info as input: same deck info means same dealer possibilities
+ after getting each possible score from deck info, calculate win lose and push ratios for each player hand
+ (same deck info mean same hand? then even better)
5. for each drawn hand starting from the back, determine the best move (hit or stand)
+ save best moves for possible combinations, use them on all possible instead of recaculating
+ dont include obvious cases (e.g. stand on 21, hit under 12)
6. determine expected return for hands as far back as you like (start of game, dealt hands, dealer plays)

In [234]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]
ACTIVE_COLUMNS = ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS]
GAME_COLUMNS = ["chance"] + ACTIVE_COLUMNS + DECK_COLUMNS

# SETUP

In [235]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS, dtype=np.int8)
    deck.loc["10"] *= 4
    return deck

In [236]:
def order_game_columns(game):
    """order the columns of the game dataframe"""

    return game[GAME_COLUMNS + [colname for colname in game.columns if colname not in GAME_COLUMNS]]

In [237]:
def player_draws_single(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [238]:
def dealer_draws_single(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [239]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = player_draws_single(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = dealer_draws_single(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

# PLAYER

In [240]:
def player_draws_once(game, keep_drawn_column=False):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # aggregate same hands
    cols = next_game.columns.tolist()
    cols.remove("chance")
    next_game = next_game.groupby(cols).agg({'chance': 'sum'}).reset_index()
    # NOTE: is drawn column is kept, the hands will not be grouped (useful for player decisions)

    # sort columns
    next_game = order_game_columns(next_game)

    return next_game.sort_values("player_score")


In [241]:
def player_plays_all_possible(game):
    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22) & (game["player_score"] <= LAST_CHECKED_SCORE)
    playable = game[is_playable].copy()
    final = game.copy()

    # print("calculating all possible moves...")
    while len(playable) > 0:
        next_game = player_draws_once(playable, keep_drawn_column=True)
        drawnsum = len([i for i in next_game.columns.tolist() if "drawn" in i])
        next_game = next_game.rename(columns={"drawn":"drawn_"+str(drawnsum)})
        final["drawn_"+str(drawnsum)] = np.zeros(len(final)).astype(int)
        
        final = pd.concat((final, next_game), axis=0, ignore_index=True)

        is_playable = (next_game["player_score"] >= 0) & (next_game["player_score"] != 22) & (next_game["player_score"] <= LAST_CHECKED_SCORE)

        playable = next_game[is_playable].copy()
    
    return final

# DEALER

In [242]:
def dealer_draws_once(game, keep_drawn_column=False):
    """get updated game when dealer hits once"""
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["dealer_score"] += next_game["drawn"]
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # group and reorder
    cols = next_game.columns.tolist()
    cols.remove("chance")
    next_game = next_game.groupby(cols).agg({'chance': 'sum'}).reset_index()

    return next_game.sort_values("dealer_score")


In [243]:
def dealer_plays(game):
    """get outcomes for dealer play with returns, grouped by original deck info"""
    
    assert set(DECK_COLUMNS).issubset(set(game.columns.tolist())), "Deck Info not in columns"
    assert set(["dealer_score","dealer_aces","dealer_cards"]).issubset(game.columns.tolist()), "not all info in columns"
    assert "initial_samedeck" in game.columns, "initial same deck column not in game"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    while len(playable) > 0:
        next_game = dealer_draws_once(playable)

        is_playable = (next_game["dealer_score"] < 17) & (next_game["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_game[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_game[is_playable].copy()

    final = final.groupby(["dealer_score","initial_samedeck"]).agg({"chance":"sum"}).reset_index()

    return final

In [244]:
def dealer_get_expected_return(game, verbose=False):
    """simulates dealer play, adds column "expected_return" """

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # TODO: get only distinct decks (dealer and deck)
    # get the expected outputs for each distinct
    # merge them back onto the game, calculate returns

    game["initial_samedeck"] = game.groupby(["dealer_score","dealer_cards","dealer_aces"] + DECK_COLUMNS).ngroup()

    distinct_decks = game[["initial_samedeck","dealer_score","dealer_cards","dealer_aces"] + DECK_COLUMNS].drop_duplicates().copy()
    # could drop by "initial_samdeck" for performance

    print(f"Simplified from {len(game)} to {len(distinct_decks)} games")
    print("distinct decks\n", distinct_decks)

    distinct_decks["chance"] = 1.0
    dealer_scores = dealer_plays(distinct_decks)
    dealer_scores = dealer_scores.rename(columns={"chance":"normed_chance"})

    dealer_scores = pd.merge(dealer_scores, pd.DataFrame({"player_score":np.arange(4,23)}), how="cross") # add possible player scores, is this really faster than calculating it later?
    dealer_scores = get_return_column(dealer_scores)
    dealer_scores["expected_return"] = dealer_scores["normed_chance"] * dealer_scores["return"]

    print("dealer with er\n",dealer_scores)
    dealer_scores_grouped = dealer_scores.groupby(["player_score","initial_samedeck"]).agg({"expected_return":"sum","normed_chance":"sum"}).reset_index()

    print("dealer_scores_grouped\n", dealer_scores_grouped.sort_values("initial_samedeck"))

    # merge back to original game
    game = pd.merge(game, dealer_scores_grouped.loc[:, ["player_score","initial_samedeck","expected_return"]], on=["initial_samedeck","player_score"], how="left")
    game["expected_return"] *= game["chance"]

    group_cols = game.columns.tolist()
    group_cols.remove("expected_return")
    group_cols.remove("chance")

    # what is this good for? everything should be grouped already
    # game = game.groupby(group_cols).agg({"expected_return":"sum","chance":"sum"}).reset_index()

    print("updated game\n", game)

    return game


# RETURNS

In [245]:
def get_expected_return_standing(game, verbose=False):
    """add column "expected return" to game,
    player has minimum of 2 cards, dealer has 1 upcard
    assumes perfect play"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # bust hands
    is_bust = game["player_score"] == -2
    finished = game[is_bust].copy()
    playable = game[~is_bust].copy()

    if verbose:
        print("Calculating possible returns for each game...")
        print(f"Bust games: {len(finished) / len(game) * 100:.2f}%")

    dealer_outcomes = dealer_get_expected_return(playable, verbose)

    finished["expected_return"] = -finished["chance"]
    
    final = pd.concat((finished, dealer_outcomes), axis=0, ignore_index=True)

    return final


In [246]:
def get_return_column(game):
    """add "return" column to game dataframe, denoting the return of the game"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert "player_score" in game.columns, "no player score present"

    game["return"] = pd.Series(np.zeros(len(game)))

    game.loc[(game["player_score"] == 22) & (game["dealer_score"] != 22), "return"] = 1.5
    game.loc[(game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22), "return"] = 1
    game.loc[game["player_score"] == game["dealer_score"], "return"] = 0
    game.loc[game["player_score"] < game["dealer_score"], "return"] = -1

    return game

In [247]:
def get_expected_return_best(game, verbose=False):
    """add column "expected_return" to game dataframe
    assumes best play"""

    assert np.all(game["player_cards"] >= 2), "player has less than 2 cards"
    assert np.all(game["dealer_cards"] == 1), "dealer has no cards showing"

    print("game\n", game.sort_values(["player_score","dealer_score"]))

    t1 = time.time()

    game["initial"] = np.arange(len(game))

    if np.all(game["player_score"] == 22):
        return get_expected_return_standing(game)

    final = player_plays_all_possible(game)
    print("player played\n",final)
    final = get_expected_return_standing(final, verbose)
    final["stand_return"] = np.nan
    final["hit_return"] = np.nan

    drawn_columns = [i for i in final.columns.tolist() if "drawn" in i]

    print("----- playing", time.time()-t1)
    
    print("final sorted by cards\n",final.sort_values(["player_cards"] + drawn_columns))

    # get double scores if player has 2 cards and can draw at least once 
    if np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" in drawn_columns:
        if verbose:
            print("double vals\n", final.loc[(final["drawn_1"] != 0) & (final["drawn_2"] == 0), [col for col in final.columns if "drawn" not in col]+["drawn_1"]])
        double_return = final.loc[(final["drawn_1"] != 0) & (final["drawn_2"] == 0), "expected_return"].sum() * 2
    elif np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" not in drawn_columns:
        double_return = final.loc[(final["drawn_1"] != 0), "expected_return"].sum() * 2
    else:
        double_return = None

    for i in range(len(drawn_columns)):
        col = drawn_columns[len(drawn_columns)-i-1]
        cols = drawn_columns[:len(drawn_columns)-i-1]
        print(col)
        print("----- sim",col, time.time()-t1)


        # get all games that can stand or hit in that column
        if len(cols) != 0:
            useable = (final[col] != 0) | ((final[col] == 0) & (final[cols[-1]] != 0) & (final["player_score"] > 0) & (final["player_score"] <= LAST_CHECKED_SCORE))
        else:
            useable = (final[col] != 0) | ((final[col] == 0) & (final["player_score"] > 0))
        
        used = final[useable].copy()
        used["stand"] = used[col] == 0
        print("used\n", used)

        grouped = used.groupby(cols + ["stand","initial"]).agg({'chance': 'sum', "expected_return": "sum"}).reset_index()
        print("grouped\n",grouped)

        standing_games = used[used["stand"]].drop(columns=["expected_return",col,"stand","stand_return","hit_return"])
        # here I could group by player score and cards left. stand/hit choice will stay the same, and itz will group better
        
        standing_games = pd.merge(standing_games, grouped.loc[grouped["stand"] == True, cols+["initial","expected_return"]], on=cols+["initial"], how="left")
        standing_games = standing_games.rename(columns={"expected_return":"stand_return"})
        standing_games = pd.merge(standing_games, grouped.loc[grouped["stand"] == False, cols+["initial","expected_return"]], on=cols+["initial"], how="left")
        standing_games = standing_games.rename(columns={"expected_return":"hit_return"})
        # could this be made better?
        
        standing_games["expected_return"] = np.max(standing_games[["stand_return","hit_return"]], axis=1)
        standing_games["best"] = np.where(standing_games['hit_return'] > standing_games['stand_return'], "hit", "stand")  
        print("standing_games\n", standing_games.sort_values(cols))
        
        # drop drawn column from unused games
        final = final[~useable].copy().drop(columns=[col])

        final = pd.concat([final, standing_games], axis=0, ignore_index=True)

        print(final.sort_values(["player_cards"] + cols))

    final = final.drop(columns=["initial"])
    final["normed_return"] = final["expected_return"] / final["chance"]  
    print("----- final", time.time()-t1)

    return final

In [248]:
def get_expected_return_from_any(game):
    """get the expected return of the game from any state"""

    if np.all(game["player_cards"] == 0) and np.all(game["dealer_cards"] == 0):
        print("Game is in initial state")
        game = dealer_draws_once(game)
        game = player_draws_once(game)
        game = player_draws_once(game)
        print(game)

    game = get_expected_return_best(game)

    return game

# TESTING

In [249]:
game = get_game(None, [2],[4])  
game = player_draws_once(game)  
r = get_expected_return_best(game) 
r

game
      chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0  0.074194             4             2            0             4             1            0  22  24  23  24  24  24  24  24  96  24
1  0.077419             5             2            0             4             1            0  23  23  23  24  24  24  24  24  96  24
2  0.074194             6             2            0             4             1            0  23  24  22  24  24  24  24  24  96  24
3  0.077419             7             2            0             4             1            0  23  24  23  23  24  24  24  24  96  24
4  0.077419             8             2            0             4             1            0  23  24  23  24  23  24  24  24  96  24
5  0.077419             9             2            0             4             1            0  23  24  23  24  24  23  24  24  96  24
6  0.077419            10             2            0    

player played
               chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  ...  11  initial  drawn_1  drawn_2  drawn_3  drawn_4  drawn_5  drawn_6  drawn_7  drawn_8  drawn_9  drawn_10  drawn_11  drawn_12  drawn_13
0       7.419355e-02             4             2            0             4             1            0  22  24  23  24  24  24  24  24  ...  24        0        0        0        0        0        0        0        0        0        0         0         0         0         0
1       7.741935e-02             5             2            0             4             1            0  23  23  23  24  24  24  24  24  ...  24        1        0        0        0        0        0        0        0        0        0         0         0         0         0
2       7.419355e-02             6             2            0             4             1            0  23  24  22  24  24  24  24  24  ...  24        2        0  

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,initial_samedeck,stand_return,hit_return,best,normed_return
0,0.074194,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24,-0.003470,1023.0,-0.015408,-0.003470,hit,-0.046774
1,0.077419,5,2,0,4,1,0,23,23,23,24,24,24,24,24,96,24,-0.004498,1377.0,-0.015897,-0.004498,hit,-0.058096
2,0.074194,6,2,0,4,1,0,23,24,22,24,24,24,24,24,96,24,-0.005193,1537.0,-0.015202,-0.005193,hit,-0.069987
3,0.077419,7,2,0,4,1,0,23,24,23,23,24,24,24,24,96,24,-0.002922,1617.0,-0.015833,-0.002922,hit,-0.037741
4,0.077419,8,2,0,4,1,0,23,24,23,24,23,24,24,24,96,24,0.003518,1662.0,-0.015867,0.003518,hit,0.045436
5,0.077419,9,2,0,4,1,0,23,24,23,24,24,23,24,24,96,24,0.010462,1691.0,-0.015921,0.010462,hit,0.135128
6,0.077419,10,2,0,4,1,0,23,24,23,24,24,24,23,24,96,24,0.018127,1710.0,-0.016224,0.018127,hit,0.234141
7,0.077419,11,2,0,4,1,0,23,24,23,24,24,24,24,23,96,24,0.022122,1722.0,-0.016286,0.022122,hit,0.285737
8,0.309677,12,2,0,4,1,0,23,24,23,24,24,24,24,24,95,24,-0.065145,1730.0,-0.065378,-0.065145,hit,-0.210364
9,0.077419,13,2,1,4,1,0,23,24,23,24,24,24,24,24,96,23,0.008029,1737.0,-0.016007,0.008029,hit,0.103706


In [250]:
import cProfile 
cProfile.run('get_expected_return_best(game) ', 'output.prof')  

game
      chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11  initial
0  0.074194             4             2            0             4             1            0  22  24  23  24  24  24  24  24  96  24        0
1  0.077419             5             2            0             4             1            0  23  23  23  24  24  24  24  24  96  24        1
2  0.074194             6             2            0             4             1            0  23  24  22  24  24  24  24  24  96  24        2
3  0.077419             7             2            0             4             1            0  23  24  23  23  24  24  24  24  96  24        3
4  0.077419             8             2            0             4             1            0  23  24  23  24  23  24  24  24  96  24        4
5  0.077419             9             2            0             4             1            0  23  24  23  24  24  23  24  24  96  24   

In [251]:
dealer_get_expected_return(player_draws_once(player_draws_once(dealer_draws_once(get_game()))))

Simplified from 550 to 550 games
distinct decks
      initial_samedeck  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0                   0             2             1            0  21  24  24  24  24  24  24  24  96  24
1                  55             3             1            0  22  23  24  24  24  24  24  24  96  24
2                 110             4             1            0  22  24  23  24  24  24  24  24  96  24
3                 165             5             1            0  22  24  24  23  24  24  24  24  96  24
4                 220             6             1            0  22  24  24  24  23  24  24  24  96  24
..                ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
542               163             4             1            0  24  24  23  24  24  24  24  24  95  23
541               108             3             1            0  24  23  24  24  24  24  24  24  95  23
548               493   

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,initial_samedeck,expected_return
0,0.000404,4,2,0,2,1,0,21,24,24,24,24,24,24,24,96,24,0,-0.000118
1,0.000440,4,2,0,3,1,0,22,23,24,24,24,24,24,24,96,24,55,-0.000111
2,0.000440,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24,110,-0.000091
3,0.000440,4,2,0,5,1,0,22,24,24,23,24,24,24,24,96,24,165,-0.000070
4,0.000440,4,2,0,6,1,0,22,24,24,24,23,24,24,24,96,24,220,-0.000066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,0.003677,22,2,1,4,1,0,24,24,23,24,24,24,24,24,95,23,163,0.005515
546,0.003677,22,2,1,3,1,0,24,23,24,24,24,24,24,24,95,23,108,0.005515
547,0.014553,22,2,1,10,1,0,24,24,24,24,24,24,24,24,94,23,493,0.020205
548,0.003677,22,2,1,5,1,0,24,24,24,23,24,24,24,24,95,23,218,0.005515


In [252]:
ret_full = get_expected_return_from_any(get_game())
ret_full

Game is in initial state
       chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0    0.000404             4             2            0             2             1            0  21  24  24  24  24  24  24  24  96  24
1    0.000440             4             2            0             3             1            0  22  23  24  24  24  24  24  24  96  24
2    0.000440             4             2            0             4             1            0  22  24  23  24  24  24  24  24  96  24
3    0.000440             4             2            0             5             1            0  22  24  24  23  24  24  24  24  96  24
4    0.000440             4             2            0             6             1            0  22  24  24  24  23  24  24  24  96  24
..        ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
542  0.003677          

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,initial_samedeck,stand_return,hit_return,best,normed_return
0,0.000404,4,2,0,2,1,0,21,24,24,24,24,24,24,24,96,24,-0.000046,1023.0,-0.000118,-0.000046,hit,-0.114739
1,0.000440,4,2,0,3,1,0,22,23,24,24,24,24,24,24,96,24,-0.000036,3881.0,-0.000111,-0.000036,hit,-0.082064
2,0.000440,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24,-0.000021,6739.0,-0.000091,-0.000021,hit,-0.046774
3,0.000440,4,2,0,5,1,0,22,24,24,23,24,24,24,24,96,24,-0.000002,9597.0,-0.000070,-0.000002,hit,-0.004963
4,0.000440,4,2,0,6,1,0,22,24,24,24,23,24,24,24,96,24,0.000006,12455.0,-0.000066,0.000006,hit,0.014623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,0.003677,22,2,1,4,1,0,24,24,23,24,24,24,24,24,95,23,0.005515,8565.0,0.005515,NaN,stand,1.500000
546,0.003677,22,2,1,3,1,0,24,23,24,24,24,24,24,24,95,23,0.005515,5707.0,0.005515,NaN,stand,1.500000
547,0.014553,22,2,1,10,1,0,24,24,24,24,24,24,24,24,94,23,0.020205,25713.0,0.020205,NaN,stand,1.388350
548,0.003677,22,2,1,5,1,0,24,24,24,23,24,24,24,24,95,23,0.005515,11423.0,0.005515,NaN,stand,1.500000


In [257]:
ret_full[(ret_full["player_score"] == 18) & (ret_full["player_aces"] == 1)].sort_values("dealer_score")

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,initial_samedeck,stand_return,hit_return,best,normed_return
473,0.000919,18,2,1,2,1,0,23,24,24,24,24,23,24,24,96,23,0.000114,2798.0,0.000114,0.000058,stand,0.124001
474,0.000919,18,2,1,3,1,0,24,23,24,24,24,23,24,24,96,23,0.000139,5656.0,0.000139,0.000083,stand,0.151119
475,0.000919,18,2,1,4,1,0,24,24,23,24,24,23,24,24,96,23,0.000166,8514.0,0.000166,0.000114,stand,0.180238
482,0.000919,18,2,1,5,1,0,24,24,24,23,24,23,24,24,96,23,0.000187,11372.0,0.000187,0.000140,stand,0.203128
477,0.000919,18,2,1,6,1,0,24,24,24,24,23,23,24,24,96,23,0.000258,14230.0,0.000258,0.000176,stand,0.280488
476,0.000881,18,2,1,7,1,0,24,24,24,24,24,22,24,24,96,23,0.000354,17088.0,0.000354,0.000151,stand,0.401861
478,0.000919,18,2,1,8,1,0,24,24,24,24,24,23,23,24,96,23,0.000099,19946.0,0.000099,0.000038,stand,0.108057
479,0.000919,18,2,1,9,1,0,24,24,24,24,24,23,24,23,96,23,-0.000091,22804.0,-0.000168,-0.000091,hit,-0.098469
480,0.003677,18,2,1,10,1,0,24,24,24,24,24,23,24,24,95,23,-0.000760,25662.0,-0.000885,-0.000760,hit,-0.206708
483,0.000881,18,2,1,11,1,1,24,24,24,24,24,23,24,24,96,22,-0.000332,28520.0,-0.000335,-0.000332,hit,-0.376403


In [268]:
ret_full.sort_values("chance")

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,initial_samedeck,stand_return,hit_return,best,normed_return
0,0.000404,4,2,0,2,1,0,21,24,24,24,24,24,24,24,96,24,-0.000046,1023.0,-0.000118,-0.000046,hit,-0.114739
335,0.000404,14,2,0,7,1,0,24,24,24,24,24,21,24,24,96,24,-0.000134,17058.0,-0.000193,-0.000134,hit,-0.331784
250,0.000404,12,2,0,6,1,0,24,24,24,24,21,24,24,24,96,24,-0.000062,14145.0,-0.000062,-0.000070,stand,-0.154706
419,0.000404,16,2,0,8,1,0,24,24,24,24,24,24,21,24,96,24,-0.000183,19952.0,-0.000209,-0.000183,hit,-0.453401
34,0.000404,6,2,0,3,1,0,24,21,24,24,24,24,24,24,96,24,-0.000044,4873.0,-0.000101,-0.000044,hit,-0.108263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.014553,12,2,0,10,1,0,23,24,24,24,24,24,24,24,94,24,-0.006173,24601.0,-0.008383,-0.006173,hit,-0.424152
510,0.014553,19,2,0,10,1,0,24,24,24,24,24,24,24,23,94,24,-0.000198,25693.0,-0.000198,NaN,stand,-0.013620
448,0.014553,17,2,0,10,1,0,24,24,24,24,24,23,24,24,94,24,-0.006747,25650.0,-0.006747,-0.008917,stand,-0.463626
313,0.014553,14,2,0,10,1,0,24,24,23,24,24,24,24,24,94,24,-0.007346,25419.0,-0.008381,-0.007346,hit,-0.504799


In [276]:
ret_half10.sort_values("normed_return")

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,initial_samedeck,stand_return,hit_return,best,normed_return
441,0.001520,17,2,0,11,1,1,24,24,24,24,24,24,23,23,48.0,23,-0.000887,28531.0,-0.000996,-0.000887,hit,-0.583505
440,0.003040,17,2,0,11,1,1,24,24,24,24,24,23,24,24,47.0,23,-0.001773,28508.0,-0.001991,-0.001773,hit,-0.583337
406,0.001520,16,2,0,11,1,1,24,24,24,24,24,23,24,23,48.0,23,-0.000828,28503.0,-0.001245,-0.000828,hit,-0.544896
408,0.000728,16,2,0,11,1,1,24,24,24,24,24,24,22,24,48.0,23,-0.000397,28526.0,-0.000596,-0.000397,hit,-0.544784
400,0.003040,16,2,0,11,1,1,24,24,24,24,23,24,24,24,47.0,23,-0.001655,28468.0,-0.002490,-0.001655,hit,-0.544466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,0.003040,22,2,1,7,1,0,24,24,24,24,24,23,24,24,47.0,23,0.004560,17139.0,0.004560,NaN,stand,1.500000
543,0.003040,22,2,1,6,1,0,24,24,24,24,23,24,24,24,47.0,23,0.004560,14281.0,0.004560,NaN,stand,1.500000
544,0.003040,22,2,1,2,1,0,23,24,24,24,24,24,24,24,47.0,23,0.004560,2849.0,0.004560,NaN,stand,1.500000
545,0.003040,22,2,1,4,1,0,24,24,23,24,24,24,24,24,47.0,23,0.004560,8565.0,0.004560,NaN,stand,1.500000


In [270]:
ret_full[ret_full["player_score"] == 22]["chance"].sum()

0.04748948800395747

In [267]:
ret_full["expected_return"].describe()

count    550.000000
mean      -0.000043
std        0.001683
min       -0.008307
25%       -0.000326
50%       -0.000097
75%        0.000110
max        0.020205
Name: expected_return, dtype: float64

In [265]:
ret_full["best"].value_counts()

hit      369
stand    181
Name: best, dtype: int64

In [280]:
ret_full["expected_return"].sum()

-0.023435236711825444

In [284]:

ret_half10["expected_return"].sum()

0.02250369949362703

In [282]:
half10game = get_game()
half10game["10"] *= 2
ret_half10 = get_expected_return_from_any(half10game)
ret_half10

Game is in initial state
       chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9   10  11
0    0.000180             4             2            0             2             1            0  21  24  24  24  24  24  24  24  192  24
1    0.000197             4             2            0             3             1            0  22  23  24  24  24  24  24  24  192  24
2    0.000197             4             2            0             4             1            0  22  24  23  24  24  24  24  24  192  24
3    0.000197             4             2            0             5             1            0  22  24  24  23  24  24  24  24  192  24
4    0.000197             4             2            0             6             1            0  22  24  24  24  23  24  24  24  192  24
..        ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..
542  0.003281   

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,initial_samedeck,stand_return,hit_return,best,normed_return
0,0.000180,4,2,0,2,1,0,21,24,24,24,24,24,24,24,192,24,0.000013,1023.0,-0.000008,0.000013,hit,0.073393
1,0.000197,4,2,0,3,1,0,22,23,24,24,24,24,24,24,192,24,0.000021,3881.0,-0.000002,0.000021,hit,0.105199
2,0.000197,4,2,0,4,1,0,22,24,23,24,24,24,24,24,192,24,0.000027,6739.0,0.000005,0.000027,hit,0.138515
3,0.000197,4,2,0,5,1,0,22,24,24,23,24,24,24,24,192,24,0.000034,9597.0,0.000014,0.000034,hit,0.175337
4,0.000197,4,2,0,6,1,0,22,24,24,24,23,24,24,24,192,24,0.000037,12455.0,0.000015,0.000037,hit,0.187927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,0.003281,22,2,1,4,1,0,24,24,23,24,24,24,24,24,191,23,0.004921,8565.0,0.004921,NaN,stand,1.500000
546,0.003281,22,2,1,3,1,0,24,23,24,24,24,24,24,24,191,23,0.004921,5707.0,0.004921,NaN,stand,1.500000
547,0.026109,22,2,1,10,1,0,24,24,24,24,24,24,24,24,190,23,0.036940,25713.0,0.036940,NaN,stand,1.414815
548,0.003281,22,2,1,5,1,0,24,24,24,23,24,24,24,24,191,23,0.004921,11423.0,0.004921,NaN,stand,1.500000


In [283]:
ret_half10[ret_half10["player_score"] == 22]["chance"].sum()

0.05549934961699667

## POSSIBLE SPEEDUPS

- [MUST] when calculating dealer return, calculate the return for each possible pairing of dealer and player score beforehand. Instead of merging df and then calculating for each hand 
- [MUST] for player moves, calculate bust chance and if bust chance is greater than current return, then don't bother calculating hit moves
- [MAYBE] simplify merges by creating a custom merge column first, instead of merging on e.g. all drawn columns (np unique, merge)
- [MAYBE] merge hands even if they have to split again later on

get_expected_return_from_any(get_game())  
*20:52 min*  
*1:09 min* calc only <= 17

game = get_game(None, [2],[4])  
game = player_draws_once(game)  
get_expected_return_best(game)    
*0:33 min*  
*0:07 min* calc only <= 17  
*6.6 sec* ngroup() unique

In [254]:
pos = player_plays_all_possible(game)
pos[pos["player_score"] > 0].sort_values("player_score")

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,...,11,initial,drawn_1,drawn_2,drawn_3,drawn_4,drawn_5,drawn_6,drawn_7,drawn_8,drawn_9,drawn_10,drawn_11,drawn_12,drawn_13
0,7.419355e-02,4,2,0,4,1,0,22,24,23,24,24,24,24,24,...,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7.741935e-02,5,2,0,4,1,0,23,23,23,24,24,24,24,24,...,24,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7.419355e-02,6,2,0,4,1,0,23,24,22,24,24,24,24,24,...,24,2,0,0,0,0,0,0,0,0,0,0,0,0,0
11,5.282389e-03,6,3,0,4,1,0,21,24,23,24,24,24,24,24,...,24,0,2,0,0,0,0,0,0,0,0,0,0,0,0
12,5.762606e-03,7,3,0,4,1,0,22,23,23,24,24,24,24,24,...,24,0,3,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99587,1.045480e-09,21,9,0,4,1,0,21,23,22,23,24,24,24,24,...,21,0,2,11,11,4,3,11,5,0,0,0,0,0,0
99588,1.045480e-09,21,9,0,4,1,0,21,23,22,23,24,24,24,24,...,21,0,2,5,11,4,11,11,3,0,0,0,0,0,0
99589,1.045480e-09,21,9,0,4,1,0,21,23,22,23,24,24,24,24,...,21,0,2,5,11,3,11,11,4,0,0,0,0,0,0
99559,1.045480e-09,21,9,0,4,1,0,21,23,22,23,24,24,24,24,...,21,2,2,2,3,11,11,11,5,0,0,0,0,0,0


In [255]:
get_expected_return_standing(game)

Simplified from 10 to 10 games
distinct decks
    initial_samedeck  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0                 0             4             1            0  22  24  23  24  24  24  24  24  96  24
1                 1             4             1            0  23  23  23  24  24  24  24  24  96  24
2                 2             4             1            0  23  24  22  24  24  24  24  24  96  24
3                 3             4             1            0  23  24  23  23  24  24  24  24  96  24
4                 4             4             1            0  23  24  23  24  23  24  24  24  96  24
5                 5             4             1            0  23  24  23  24  24  23  24  24  96  24
6                 6             4             1            0  23  24  23  24  24  24  23  24  96  24
7                 7             4             1            0  23  24  23  24  24  24  24  23  96  24
8                 8             4           

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,initial,expected_return,initial_samedeck
0,0.074194,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24,0,-0.015408,0.0
1,0.077419,5,2,0,4,1,0,23,23,23,24,24,24,24,24,96,24,1,-0.015897,1.0
2,0.074194,6,2,0,4,1,0,23,24,22,24,24,24,24,24,96,24,2,-0.015202,2.0
3,0.077419,7,2,0,4,1,0,23,24,23,23,24,24,24,24,96,24,3,-0.015833,3.0
4,0.077419,8,2,0,4,1,0,23,24,23,24,23,24,24,24,96,24,4,-0.015867,4.0
5,0.077419,9,2,0,4,1,0,23,24,23,24,24,23,24,24,96,24,5,-0.015921,5.0
6,0.077419,10,2,0,4,1,0,23,24,23,24,24,24,23,24,96,24,6,-0.016224,6.0
7,0.077419,11,2,0,4,1,0,23,24,23,24,24,24,24,23,96,24,7,-0.016286,7.0
8,0.309677,12,2,0,4,1,0,23,24,23,24,24,24,24,24,95,24,8,-0.065378,8.0
9,0.077419,13,2,1,4,1,0,23,24,23,24,24,24,24,24,96,23,9,-0.016007,9.0


In [256]:
get_expected_return_best(game)

game
      chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11  initial
0  0.074194             4             2            0             4             1            0  22  24  23  24  24  24  24  24  96  24        0
1  0.077419             5             2            0             4             1            0  23  23  23  24  24  24  24  24  96  24        1
2  0.074194             6             2            0             4             1            0  23  24  22  24  24  24  24  24  96  24        2
3  0.077419             7             2            0             4             1            0  23  24  23  23  24  24  24  24  96  24        3
4  0.077419             8             2            0             4             1            0  23  24  23  24  23  24  24  24  96  24        4
5  0.077419             9             2            0             4             1            0  23  24  23  24  24  23  24  24  96  24   

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,initial_samedeck,stand_return,hit_return,best,normed_return
0,0.074194,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24,-0.003470,1023.0,-0.015408,-0.003470,hit,-0.046774
1,0.077419,5,2,0,4,1,0,23,23,23,24,24,24,24,24,96,24,-0.004498,1377.0,-0.015897,-0.004498,hit,-0.058096
2,0.074194,6,2,0,4,1,0,23,24,22,24,24,24,24,24,96,24,-0.005193,1537.0,-0.015202,-0.005193,hit,-0.069987
3,0.077419,7,2,0,4,1,0,23,24,23,23,24,24,24,24,96,24,-0.002922,1617.0,-0.015833,-0.002922,hit,-0.037741
4,0.077419,8,2,0,4,1,0,23,24,23,24,23,24,24,24,96,24,0.003518,1662.0,-0.015867,0.003518,hit,0.045436
5,0.077419,9,2,0,4,1,0,23,24,23,24,24,23,24,24,96,24,0.010462,1691.0,-0.015921,0.010462,hit,0.135128
6,0.077419,10,2,0,4,1,0,23,24,23,24,24,24,23,24,96,24,0.018127,1710.0,-0.016224,0.018127,hit,0.234141
7,0.077419,11,2,0,4,1,0,23,24,23,24,24,24,24,23,96,24,0.022122,1722.0,-0.016286,0.022122,hit,0.285737
8,0.309677,12,2,0,4,1,0,23,24,23,24,24,24,24,24,95,24,-0.065145,1730.0,-0.065378,-0.065145,hit,-0.210364
9,0.077419,13,2,1,4,1,0,23,24,23,24,24,24,24,24,96,23,0.008029,1737.0,-0.016007,0.008029,hit,0.103706
